In [79]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping
from keras_preprocessing import image
from sklearn.model_selection import train_test_split
from pathlib import Path
ROOT = "/Users/alistairwestwood/Documents/AML/AMLS_assignment22_23/AMLS_22-23_SN19043423" #Root directory is the folder this file is placed in

train_dir = os.path.join(ROOT,'Datasets/cartoon_set/img')
labels_train = os.path.join(ROOT,'Datasets/cartoon_set/labels.csv')
test_dir = os.path.join(ROOT,'Datasets/cartoon_set_test/img')
labels_test = os.path.join(ROOT, 'Datasets/cartoon_set_test/labels.csv')

In [80]:
def importData(img_dir, labels_dir):
    image_paths = np.array([os.path.join(img_dir, l) for l in os.listdir(img_dir)])
    labels_file = pd.read_csv(os.path.join(ROOT, labels_dir), sep = '\t', engine = 'python', header = 0)
    face_labels_df = labels_file['eye_color']
    face_labels = face_labels_df.values
    face_imgs = []
    for i in image_paths:
        img = tf.io.read_file(i)
        img = tf.image.decode_png(img, channels= 3)
        img = tf.image.resize(img, [100, 100])
        face_imgs.append(img)
    X = np.array(face_imgs)
    print("Data imported")

    return X, face_labels

In [89]:
def CNNmodel():
    model = Sequential()
    model.add(Flatten(input_shape = (100,100,3)))
    model.add(Dense(350, activation = 'relu'))
    model.add(Dense(5, activation = 'softmax'))

    return model

In [82]:
train_imgs, train_labels = importData(train_dir, labels_train)
test_imgs, test_labels = importData(test_dir, labels_test)

Data imported
Data imported


In [83]:
train_labels.dtype

dtype('int64')

In [90]:
dataset_train = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))
dataset_train = dataset_train.batch(32)
dataset_train = dataset_train.map(lambda x, y: (x/255, y))

In [91]:
ds_size = len(dataset_train)
train_size = int(ds_size * 0.8)

Train = dataset_train.take(train_size)
Val = dataset_train.skip(train_size)

In [92]:
early_stop = EarlyStopping(monitor = "val_loss", restore_best_weights=True, patience=5, verbose=1)
callback = [early_stop]

In [93]:
model = CNNmodel()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 30000)             0         
                                                                 
 dense_24 (Dense)            (None, 350)               10500350  
                                                                 
 dense_25 (Dense)            (None, 5)                 1755      
                                                                 
Total params: 10,502,105
Trainable params: 10,502,105
Non-trainable params: 0
_________________________________________________________________


In [94]:
history = model.fit(Train, epochs=1000, validation_data=Val, callbacks=callback)

Epoch 1/1000
250/250 [==============================] - 33s 130ms/step - loss: 5.8049 - accuracy: 0.1996 - val_loss: 1.6607 - val_accuracy: 0.2035
Epoch 2/1000
250/250 [==============================] - 32s 127ms/step - loss: 2.0748 - accuracy: 0.1984 - val_loss: 1.7524 - val_accuracy: 0.2070
Epoch 3/1000
172/250 [===================>..........] - ETA: 9s - loss: 1.9173 - accuracy: 0.1988

KeyboardInterrupt: 